# Picogreen DNA Quantification Analysis Automation
#### [By Joon Hwan Hong](https://github.com/Joon-Hwan-Hong "My Github Page!")

## Introduction
Picogreen DNA quantification protocol is a method to quantify DNA in samples. DNA samples were obtained from post-mortem human brain tissue at the Douglas Mental Health University Institute. Samples were diluted and placed into 96-well plates, which was quantified in the TECAN Spark reader using a modified PicoGreen protocol.

I made this notebook as it was annoying to do it by hand every time on excel. Saves time and why not make it as a personal project in using pandas and excel manipulations. This notebook accounts for the plate layout including triple copy of the Standard DNA curve and the DNA sample slots. (3 wells per sample/DNA curve). Currently the notebook requires that the plate layout file and the results sheet to be on the same directory, will be improved later. This notebook is in no way optimized nor elegant.

## How to use
1. Put this jupyter notebook and two separate excel files (one generated from TECAN Spark reader for Picogreen and the protocol plate layout) into the same directory/ folder.
> Example of plate layout is shown below and on the repository. Position of standard DNA curve chosen arbitrarily
> ![alt text](Layout_Example.png "Layout_Example")
4. #### [CLICK HERE](#section2) to change settings (need to change the name of layout and tecan excel file) 
3. Click "Cell" on the Menu bar, then click "Run All"
4. The report should be generated in the same directory / folder

### Function & Development Plan
The notebook achieves the following:
1. Display plate layout and the individual quantification recordings of each 96 well slots 
2. Determine the average value for each DNA sample
3. Determine the concentration for each DNA sample
4. Visualize the data obtained
5. Create a new summarized excel sheet containing all the data, and maybe inclue metadata from the results sheet as well, visualizations in one nice excel file with multiple sheets (Partial)
6. Detect extreme/ "odd recordings" and exclude from calculations (TBD)

If there are any questions on the code, feel free to [email me](mailto:joon.hong@mail.mcgill.ca)


<h2>library imports & functions & variables needed</h2>

In [ ]:
import pandas as pd
import xlsxwriter
from openpyxl import load_workbook
import os
import sys
from datetime import datetime

# TODO: make the entire sample selections from the dna1-8 to s1-24 entirely changable instead of hard-coded

# TODO: Xlsxwritter has 2 issues: Write-only and pd can not read functions from it. Rewrite in Openxlsx


# load plate design
# coord consists of 2 integers representing row values (row1, row2)

def load_spreadsheet(file_name, sheet_name='Sheet1', coord=[]):
    # TODO: also add option for xls later? (low prio)
    spreadsheet_file = pd.ExcelFile(f'{file_name}.xlsx')
    dataframe = spreadsheet_file.parse(sheet_name)
    return dataframe if not coord else dataframe.iloc[coord[0]:coord[1],0:2]

def find_avg(assigned_pos, valid_reading=3):
    """
    return dictionary comprehension. Value selects via iloc the 3 specific rows the
    sample is placed in the TECAN reader. Summate then divide by the # of valid readings
    (which is normally 3) to obtain the average for each sample.
    """
    # TODO: impliment changes to calculation when one or more of the three readings are incorrect
    return {key: plate_df.iloc[value[0]-58:value[1]-58,0:2]['value'].sum()/valid_reading 
            for key, value in assigned_pos.items()}

def transfer_data(data_frame, xlsx_name, sheet_title):
    """
    Transfers existing panda dataframe to an Excel workbook. Adds a new sheet with desired name. 
    assumes sheet_title is a string, xlsx_name does not include file type, and both this script
    and workbook in same directory
    """
    # TODO: later add so that its not always just relative path on same directory
    relative_path = os.path.join(sys.path[0], f'{xlsx_name}.xlsx')
    book = load_workbook(relative_path)
    with pd.ExcelWriter(relative_path, engine='openpyxl') as writer:
        writer.book = book
        data_frame.to_excel(writer, sheet_name=sheet_title)
        writer.save()
        writer.close()
        
def calculate(report_name, sheet_title='Sample_Averages',calc_type='concentration', dilution_factor=500):
    """
    Currently the only function is to calculate concentration in ng/ul
    after accounting for dilution factor.
    """
    # TODO: expand for multiuse later?
    wb = openpyxl.load_workbook(f'{report_name}.xlsx')
    selected_sheet = wb[sheet_title]
    if calc_type == 'concentration':
        # for Concentration in ng/ul including dilution factor
        selected_sheet['C1'] = 'C (ng/ul)'
        for cell_obj in selected_sheet['C2:C33']:
            cell_obj[0].value = f'=0.001*{dilution_factor}*(INDIRECT("RC[-1]",0)-std_dna!B13)/(std_dna!A13)'
        # generate max and min
        selected_sheet['E1'] = 'Max:'
        selected_sheet['E2'] = 'Min:'
        selected_sheet['F1'] = '=MAX(C2:C33)'
        selected_sheet['F2'] = '=MIN(C2:C33)'
    wb.save(f'{report_name}.xlsx')
    
def generate_title(exp_name,plate_num,pop_num):
    return datetime.now().strftime('%Y-%m-%d_%Hh_%Mm_%Ss')+f'_{exp_name}_Plate{plate_num}_Pop{pop_num}_report'



<a id='section2'></a>
## Misc. Options and settings

Edit & run this cell if you wish to change some settings in the notebook

In [408]:
# File Names
results_name = 'DATA.PicoGreen.Plate1.Rep1'
plate_layout_name = 'Automation.Plate_Layout'

# can remove this option if user does not like it
pd.set_option('display.max_rows', None)

# for naming
exp_name = 'Picogreen'
plate_num = '1'
pop_num ='1'
report_name = generate_title(exp_name,plate_num,pop_num)

# alter this to change concentration calculation and graph generation
DNA_concentrations = [200,100,50,25,12.5,6.25,3.125,0]

# The assigned positions from TECAN reader to a plate
# TODO: probabily a better way to represent this mess
# TODO: make the position of the standard DNA curve modular not hard coded
layout_to_tecan24 = {
    'dna1': (58,61),   'dna2': (70,73),   'dna3': (82,85),   'dna4': (94,97), 
    'dna5': (106,109), 'dna6': (118,121), 'dna7': (130,133), 'dna8': (142,145),
    's1': (61,64),     's2': (73,76),     's3': (85,88),     's4': (97,100),
    's5': (109,112),   's6': (121,124),   's7': (133,135),   's8': (145,148),
    's9': (64,67),     's10': (76,79),    's11': (88,91),    's12': (100,103),
    's13': (112,115),  's14': (124,127),  's15': (136,139),  's16': (148,151),
    's17': (67,70),    's18': (79,82),    's19': (91,94),    's20': (103,105),
    's21': (115,118),  's22': (127,130),  's23': (139,142),  's24': (151,154)
}


<h2>96-well Plate layout</h2>

In [403]:
layout_df = load_spreadsheet(plate_layout_name)
layout_df

,1,2,3,4,5,6,7,8,9,10,11,12
A,200.000,200.000,200.000,10,10,10,48,48,48,133,133,133
B,100.000,100.000,100.000,11,11,11,55,55,55,134,134,134
C,50.000,50.000,50.000,12,12,12,58,58,58,135,135,135
D,25.000,25.000,25.000,15,15,15,67,67,67,142,142,142
E,12.500,12.500,12.500,17,17,17,84,84,84,149,149,149
F,6.250,6.250,6.250,20,20,20,104,104,104,150,150,150
G,3.125,3.125,3.125,31,31,31,118,118,118,167,167,167
H,0.000,0.000,0.000,36,36,36,127,127,127,173,173,173


<h2>Display values for each well</h2>
This will be used later to see if any wells have inconsistent values to rule out (potentially from errors in picogreen loading or sample error), an automatic function/class for this will be implimented later 

In [409]:
plate_df = load_spreadsheet(results_name, sheet_name='Result sheet', coord=[56,152])

# for pretty printing <3
plate_df.columns = ['<>','value']
plate_df.index = plate_df.index -55

plate_df


,<>,value
1,A1,48404
2,A2,47516
3,A3,49171
4,A4,2430
5,A5,2714
6,A6,2503
7,A7,4775
8,A8,4945
9,A9,5035
10,A10,1974


<h2>Calculate averages of each samples</h2>
 

In [405]:
# find average for every sample
averages = pd.DataFrame.from_dict(find_avg(layout_to_tecan24),orient='index')
averages.columns = ['value']

averages

,value
dna1,48363.666667
dna2,24551.666667
dna3,12888.666667
dna4,6818.000000
dna5,3732.333333
dna6,2045.666667
dna7,1265.666667
dna8,222.333333
s1,2549.000000
s2,223.333333


<h2>Determine Concentration of All Samples & Report Generation</h2>
This can vary from how the sample was prepared and shuld be modified if needed be.
Unfortunately the Xlsxwritter library I used initially only accepts lists(?) as input and had to convert them back.

I wanted to do this in Excel, (although matplotlib would have sufficed probably) so that I can just generate a final report in excel in the end.

>NOTE: After I began encountering issues and general limitations of xlsxwritter library, further code was written with openpyxl. I honestly do not want to rewrite these code over to the other library so its staying. Unfortunately I have come to realize the library I have been using only writes, and does not read. And it appears that [Xlsxwritter does not evaluate or write the formula](https://xlsxwriter.readthedocs.io/faq.html) result, and Pandas can not directly read it, showing 0. Latter code uses openpyxl to fix this issue.

<a id='section1'></a>
### Run this code cell to generate report
This assumes previous code cells were ran

In [406]:
# initialization setup and formatting
workbook = xlsxwriter.Workbook(f'{report_name}.xlsx')
worksheet = workbook.add_worksheet('std_dna')
bold = workbook.add_format({'bold': 1})

# Input data in excel
headings = ['Concentration','Average Value']
data = [DNA_concentrations, averages.iloc[0:8,:]['value'].tolist()]
worksheet.write_row('A1', headings, bold)
worksheet.write_column('A2', data[0])
worksheet.write_column('B2', data[1])

# create new scatter chart and add/ configure series
std_dna_linear_chart = workbook.add_chart({'type': 'scatter'})
std_dna_linear_chart.add_series({
    'name': '=std_dna!$B$1',
    'categories': '=std_dna!$A$2:$A$7',
    'trendline': {'type': 'linear', 'display_equation': True, 'line': {'color': 'red'}},
    'values': '=std_dna!$B$2:$B$7'})

# labeling, title, chart style. Then insert into worksheet
std_dna_linear_chart.set_title({'name': 'Average Picogreen Value'})
std_dna_linear_chart.set_x_axis({'name': 'Concentration (ng/ml)'})
std_dna_linear_chart.set_y_axis({'name': 'Value'})
std_dna_linear_chart.set_style(11)
worksheet.insert_chart('D2', std_dna_linear_chart, {'x_offset': 25, 'y_offset': 10})

# Record the slope and intercept needed to solve for X
worksheet.write(11,0, 'Slope')
worksheet.write(11,1, 'Intercept')
worksheet.write_formula('A13', '=SLOPE(B2:B9,A2:A9)')
worksheet.write_formula('B13', '=INTERCEPT(B2:B9,A2:A9)')
# A13 and B13 are coordinates for slope and intercept

workbook.close() 

# Create sheets containing information on layout used and recorded plate values & calculated averages
transfer_data(layout_df, report_name, 'plate_layout')
transfer_data(plate_df, report_name, 'Picogreen_Values')
transfer_data(averages, report_name, 'Sample_Averages')


# Calculate concentration from Standard DNA correlation Trendline
# The trendline will be reversed to X = (Y-B/M) to solve for concentration for all the picogreen values
calculate(report_name)